In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle

/home/richard/miniconda3/envs/esm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the absolute path of the current notebook
notebook_path = os.path.abspath('')

# Navigate to the project root (CS182-Final-Project)
project_root = os.path.dirname(notebook_path)
os.chdir(project_root)

# Now you can load the data using relative path from project root
print("Loading full dataset...")
full_data = pd.read_pickle('./data/benchmarkingGS_v1-0_similarityMeasure_sequence_v3-1.pkl')
display(full_data)


# Keep only the required columns
columns_to_keep = ['uniprotID_A', 'uniprotID_B', 'isInteraction', 'trainTest', 'sequence_A', 'sequence_B']
data = full_data[columns_to_keep]
display(data)

# Calculate dataset statistics
print("\n--- Full Dataset Statistics ---")

Loading full dataset...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,RNAseqHPA,tissueHPA,tissueCellHPA,subcellularLocationHPA,bioProcessUniprot,cellCompUniprot,molFuncUniprot,domainUniprot,motifUniprot,Bgee,sequence_A,sequence_B
0,P28223,P41595,1,test2,0.160188,-0.449930,-0.060381,NaN,0.400892,0.404061,0.680414,0.0,0.790569,0.422078,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,0.825131,0.851690,0.675880,0.000000,0.190693,0.200000,0.353553,0.0,0.000000,0.922975,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,0.930790,0.954869,0.911887,0.000000,0.547723,0.365148,0.408248,0.0,0.000000,NaN,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,0.219384,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,0.500558,-0.311704,-0.371061,0.000000,0.169811,0.375000,0.157895,0.0,0.000000,NaN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268494,O95678,Q8IYX3,0,test2,0.022646,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MSRQSSITFQSGSRRGFSTTSAITPAAGRSRFSSVSVARSAAGSGG...,MARCRHHSGYLADDEASHSMCSARVQLPKKPLVPEMRPACKPGRVP...
268495,O95835,Q8WUJ0,0,test2,0.854290,NaN,NaN,NaN,0.000000,0.447214,0.000000,0.0,0.000000,0.930655,MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...,MEDVKLEFPSLPQCKEDAEEWTYPMRREMQEILPGLFLGPYSSAMK...
268496,P60409,Q9NUB4,0,test2,0.894159,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MAASTMSVCSSDLSYGSRVCLPGSCDSCSDSWQVDDCPESCCEPPC...,MTRLCLPRPEAREDPIPVPPRGLGAGEGSGSPVRPPVSTWGPSWAQ...
268497,O43294,Q6ZRT6,0,test2,0.062636,NaN,NaN,0.288675,0.000000,0.000000,0.000000,0.0,0.000000,-0.606365,MEDLDALLSDLETTTSHMPRSGAPKERPAEPLTPPPSYGHQPQTGS...,MVSRPRSPSAFPAPWWGQQPGGPGPAKRLRLEEPAGPEPRAAPSLE...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,sequence_A,sequence_B
0,P28223,P41595,1,test2,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...
...,...,...,...,...,...,...
268494,O95678,Q8IYX3,0,test2,MSRQSSITFQSGSRRGFSTTSAITPAAGRSRFSSVSVARSAAGSGG...,MARCRHHSGYLADDEASHSMCSARVQLPKKPLVPEMRPACKPGRVP...
268495,O95835,Q8WUJ0,0,test2,MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...,MEDVKLEFPSLPQCKEDAEEWTYPMRREMQEILPGLFLGPYSSAMK...
268496,P60409,Q9NUB4,0,test2,MAASTMSVCSSDLSYGSRVCLPGSCDSCSDSWQVDDCPESCCEPPC...,MTRLCLPRPEAREDPIPVPPRGLGAGEGSGSPVRPPVSTWGPSWAQ...
268497,O43294,Q6ZRT6,0,test2,MEDLDALLSDLETTTSHMPRSGAPKERPAEPLTPPPSYGHQPQTGS...,MVSRPRSPSAFPAPWWGQQPGGPGPAKRLRLEEPAGPEPRAAPSLE...



--- Full Dataset Statistics ---


In [3]:
print("Loading full dataset...")
full_data = pd.read_pickle('data/benchmarkingGS_v1-0_similarityMeasure_sequence_v3-1.pkl')
display(full_data)


# Keep only the required columns
columns_to_keep = ['uniprotID_A', 'uniprotID_B', 'isInteraction', 'trainTest', 'sequence_A', 'sequence_B']
data = full_data[columns_to_keep]
display(data)

# Calculate dataset statistics
print("\n--- Full Dataset Statistics ---")

Loading full dataset...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,RNAseqHPA,tissueHPA,tissueCellHPA,subcellularLocationHPA,bioProcessUniprot,cellCompUniprot,molFuncUniprot,domainUniprot,motifUniprot,Bgee,sequence_A,sequence_B
0,P28223,P41595,1,test2,0.160188,-0.449930,-0.060381,NaN,0.400892,0.404061,0.680414,0.0,0.790569,0.422078,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,0.825131,0.851690,0.675880,0.000000,0.190693,0.200000,0.353553,0.0,0.000000,0.922975,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,0.930790,0.954869,0.911887,0.000000,0.547723,0.365148,0.408248,0.0,0.000000,NaN,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,0.219384,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,0.500558,-0.311704,-0.371061,0.000000,0.169811,0.375000,0.157895,0.0,0.000000,NaN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268494,O95678,Q8IYX3,0,test2,0.022646,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MSRQSSITFQSGSRRGFSTTSAITPAAGRSRFSSVSVARSAAGSGG...,MARCRHHSGYLADDEASHSMCSARVQLPKKPLVPEMRPACKPGRVP...
268495,O95835,Q8WUJ0,0,test2,0.854290,NaN,NaN,NaN,0.000000,0.447214,0.000000,0.0,0.000000,0.930655,MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...,MEDVKLEFPSLPQCKEDAEEWTYPMRREMQEILPGLFLGPYSSAMK...
268496,P60409,Q9NUB4,0,test2,0.894159,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MAASTMSVCSSDLSYGSRVCLPGSCDSCSDSWQVDDCPESCCEPPC...,MTRLCLPRPEAREDPIPVPPRGLGAGEGSGSPVRPPVSTWGPSWAQ...
268497,O43294,Q6ZRT6,0,test2,0.062636,NaN,NaN,0.288675,0.000000,0.000000,0.000000,0.0,0.000000,-0.606365,MEDLDALLSDLETTTSHMPRSGAPKERPAEPLTPPPSYGHQPQTGS...,MVSRPRSPSAFPAPWWGQQPGGPGPAKRLRLEEPAGPEPRAAPSLE...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,sequence_A,sequence_B
0,P28223,P41595,1,test2,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...
...,...,...,...,...,...,...
268494,O95678,Q8IYX3,0,test2,MSRQSSITFQSGSRRGFSTTSAITPAAGRSRFSSVSVARSAAGSGG...,MARCRHHSGYLADDEASHSMCSARVQLPKKPLVPEMRPACKPGRVP...
268495,O95835,Q8WUJ0,0,test2,MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...,MEDVKLEFPSLPQCKEDAEEWTYPMRREMQEILPGLFLGPYSSAMK...
268496,P60409,Q9NUB4,0,test2,MAASTMSVCSSDLSYGSRVCLPGSCDSCSDSWQVDDCPESCCEPPC...,MTRLCLPRPEAREDPIPVPPRGLGAGEGSGSPVRPPVSTWGPSWAQ...
268497,O43294,Q6ZRT6,0,test2,MEDLDALLSDLETTTSHMPRSGAPKERPAEPLTPPPSYGHQPQTGS...,MVSRPRSPSAFPAPWWGQQPGGPGPAKRLRLEEPAGPEPRAAPSLE...



--- Full Dataset Statistics ---


In [4]:
# Assuming your dataframe is called df
# If you need to load it:
# df = pd.read_csv('your_data.csv')

def get_unique_proteins(df):
    unique_proteins = {}
    
    for _, row in df.iterrows():
        protein_A_id = row['uniprotID_A']
        protein_A_seq = row['sequence_A']
        if protein_A_id not in unique_proteins:
            unique_proteins[protein_A_id] = protein_A_seq
            
        protein_B_id = row['uniprotID_B']
        protein_B_seq = row['sequence_B']
        if protein_B_id not in unique_proteins:
            unique_proteins[protein_B_id] = protein_B_seq
            
    return unique_proteins

train_data = pd.read_pickle('data/medium_set/train_data.pkl')
val_data = pd.read_pickle('data/medium_set/validation_data.pkl')
test1_data = pd.read_pickle('data/medium_set/test1_data.pkl')
test2_data = pd.read_pickle('data/medium_set/test2_data.pkl')

# Extract unique proteins
for df in [train_data, val_data, test1_data, test2_data]:
    unique_proteins_dict = get_unique_proteins(df)

# Print how many unique proteins were found
print(f"Found {len(unique_proteins_dict)} unique proteins")

# Save to pickle file
with open('data/medium_set/unique_proteins.pkl', 'wb') as f:
    pickle.dump(unique_proteins_dict, f)

print("Unique proteins saved to 'data/medium_set/unique_proteins.pkl'")



Found 9647 unique proteins
Unique proteins saved to 'data/medium_set/unique_proteins.pkl'


## Encode proteins with ESM C

In [5]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.cuda.is_available():
    target_gpu_index_for_fraction = 0

    print(f"Attempting to set memory fraction to 0.7 for GPU {target_gpu_index_for_fraction}")
    torch.cuda.set_per_process_memory_fraction(0.7, device=target_gpu_index_for_fraction)
    print(f"Call to set_per_process_memory_fraction for GPU {target_gpu_index_for_fraction} completed.")

model = ESMC.from_pretrained("esmc_300m").to(device)
print(f"[ESM-C] Loaded locally on {device}")
model_type = "local"

Attempting to set memory fraction to 0.7 for GPU 0
Call to set_per_process_memory_fraction for GPU 0 completed.


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 81442.80it/s]


[ESM-C] Loaded locally on cuda


In [6]:
def get_protein_embedding(sequence):
    """
    Get protein embedding for a given sequence using the loaded ESM model.
    Optimized with torch.no_grad() for inference.
    """
    with torch.no_grad():  # Ensures no gradients are computed for model operations
        protein = ESMProtein(sequence=sequence)
        protein_tensor = model.encode(protein)  # Model inference step
        logits_output = model.logits(           # Model inference step
            protein_tensor,
            LogitsConfig(sequence=False, return_embeddings=True)
        )
        # Get the per-protein representation by mean-pooling across sequence length
        embedding = logits_output.embeddings
    return embedding.cpu().to(dtype=torch.float16)


In [7]:
def embed_proteins_efficiently(proteins_dict, output_path, batch_size=100):
    """
    Efficiently embeds proteins and saves with minimal storage requirements.
    
    Args:
        proteins_dict: Dictionary of protein IDs and sequences
        output_path: Directory to save embeddings
        batch_size: Number of proteins to process in each batch
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Track proteins processed
    total_proteins = len(proteins_dict)
    proteins_processed = 0
    
    # Process proteins in batches to manage memory
    protein_ids = list(proteins_dict.keys())
    
    # Set up storage stats tracking
    total_bytes = 0
    
    # Process in batches
    for i in range(0, total_proteins, batch_size):
        batch_ids = protein_ids[i:i+batch_size]
        batch_embeddings = {}
        
        # Process each protein in the batch
        for protein_id in tqdm(batch_ids, desc=f"Processing batch {i//batch_size + 1}/{(total_proteins-1)//batch_size + 1}"):
            sequence = proteins_dict[protein_id]
            
            # Get embedding using your function that returns float16 tensors
            embedding = get_protein_embedding(sequence)
            
            # Store in batch dictionary
            batch_embeddings[protein_id] = embedding
            proteins_processed += 1
        
        # Save this batch with compression
        batch_filename = os.path.join(output_path, f"embeddings_batch_{i//batch_size + 1}.pt")
        torch.save(batch_embeddings, batch_filename, _use_new_zipfile_serialization=True)
        
        # Get file size and add to total
        batch_size_bytes = os.path.getsize(batch_filename)
        total_bytes += batch_size_bytes
        
        print(f"Saved batch {i//batch_size + 1} ({len(batch_ids)} proteins): {batch_size_bytes/1024/1024:.2f} MB")
        
        # Free up memory
        del batch_embeddings
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
    # Save a manifest file with protein IDs and their batch locations
    manifest = {protein_id: f"embeddings_batch_{i//batch_size + 1}.pt" 
                for i, protein_id in enumerate(protein_ids)}
    
    with open(os.path.join(output_path, "embeddings_manifest.pkl"), "wb") as f:
        pickle.dump(manifest, f)
    
    print(f"\nEmbedding complete: {proteins_processed} proteins processed")
    print(f"Total storage used: {total_bytes/1024/1024/1024:.2f} GB")
    print(f"Average size per protein: {total_bytes/proteins_processed/1024:.2f} KB")

def load_protein_embedding(protein_id, embeddings_dir):
    """
    Efficiently load a single protein embedding when needed.
    """
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    if protein_id not in manifest:
        raise KeyError(f"Protein ID {protein_id} not found in embeddings manifest")
    
    batch_file = manifest[protein_id]
    batch_path = os.path.join(embeddings_dir, batch_file)
    
    # Load only the specific batch file
    batch_data = torch.load(batch_path)
    
    # Return the specific embedding
    return batch_data[protein_id]

In [19]:
import gc

def embed_proteins_low_memory(proteins_dict, output_path, batch_size=10, 
                             start_batch=0, max_proteins_per_run=None,
                             max_seq_length=1500):
    """
    Low-memory implementation to embed proteins and save them efficiently.
    Can resume from a specific batch and limits proteins processed in a single run.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Track proteins processed
    total_proteins = len(proteins_dict)
    proteins_processed = 0
    
    # Process proteins in batches to manage memory
    protein_ids = list(proteins_dict.keys())
    
    # Load existing manifest if starting from a later batch
    manifest_path = os.path.join(output_path, "embeddings_manifest.pkl")
    if start_batch > 0 and os.path.exists(manifest_path):
        with open(manifest_path, "rb") as f:
            manifest = pickle.load(f)
    else:
        manifest = {}
    
    # Set up storage stats tracking
    total_bytes = 0
    
    # Apply limit if specified
    if max_proteins_per_run is not None:
        end_batch = min(start_batch + (max_proteins_per_run // batch_size) + 1, 
                       (total_proteins // batch_size) + 1)
    else:
        end_batch = (total_proteins // batch_size) + 1
    
    print(f"Will process from batch {start_batch+1} to {end_batch} (of {(total_proteins-1)//batch_size + 1} total)")
    
    # Process in batches
    total_batches = (total_proteins - 1) // batch_size + 1
    for batch_idx in range(start_batch, end_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, total_proteins)
        batch_ids = protein_ids[start_idx:end_idx]
        
        if not batch_ids:
            continue
            
        batch_filename = os.path.join(output_path, f"embeddings_batch_{batch_idx + 1}.pt")
        
        # Skip if batch already exists (for resuming)
        if os.path.exists(batch_filename):
            print(f"Batch {batch_idx + 1} already exists, skipping...")
            # Update manifest entries for this batch
            for i, protein_id in enumerate(batch_ids):
                manifest[protein_id] = f"embeddings_batch_{batch_idx + 1}.pt"
            continue
            
        batch_embeddings = {}
        
        # Process each protein in the batch individually
        for protein_id in tqdm(batch_ids, desc=f"Processing batch {batch_idx + 1}/{total_batches}"):
            sequence = proteins_dict[protein_id]
            
            # Truncate overly long sequences to reduce memory usage
            if len(sequence) > max_seq_length:
                print(f"Truncating sequence for {protein_id} from {len(sequence)} to {max_seq_length}")
                sequence = sequence[:max_seq_length]
            
            try:
                # Run embedding with explicit garbage collection before and after
                gc.collect()
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
                
                # Get embedding using your function 
                embedding = get_protein_embedding(sequence)
                
                # Move embedding to CPU immediately to free GPU memory
                embedding = embedding.cpu()
                
                # Add to batch embeddings
                batch_embeddings[protein_id] = embedding
                proteins_processed += 1
                
                # Clear memory after each protein
                gc.collect()
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
                
            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"\nOUT OF MEMORY when processing {protein_id} (len={len(sequence)})")
                    print("Skipping this protein and continuing with the next...")
                    continue
                else:
                    raise e
        
        # Save this batch if we processed any proteins
        if batch_embeddings:
            torch.save(batch_embeddings, batch_filename, _use_new_zipfile_serialization=True)
            
            # Update manifest
            for pid in batch_embeddings.keys():
                manifest[pid] = f"embeddings_batch_{batch_idx + 1}.pt"
                
            # Get file size and add to total
            batch_size_bytes = os.path.getsize(batch_filename)
            total_bytes += batch_size_bytes
            
            print(f"Saved batch {batch_idx + 1} ({len(batch_embeddings)} proteins): {batch_size_bytes/1024/1024:.2f} MB")
        
        # Free up memory
        del batch_embeddings
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        # Save manifest after each batch for safety
        with open(manifest_path, "wb") as f:
            pickle.dump(manifest, f)
    
    # Save final manifest file
    with open(manifest_path, "wb") as f:
        pickle.dump(manifest, f)
    
    print(f"\nEmbedding complete: {proteins_processed} proteins processed in this run")
    print(f"Total storage used in this run: {total_bytes/1024/1024/1024:.2f} GB")
    if proteins_processed > 0:
        print(f"Average size per protein: {total_bytes/proteins_processed/1024:.2f} KB")
    print(f"Total proteins in manifest: {len(manifest)}")

# Now run your embedding with the modified function to resume from batch 31:
with open('data/medium_set/unique_proteins.pkl', 'rb') as f:
    unique_proteins = pickle.load(f)

# Set output directory
output_dir = 'data/medium_set/embeddings'

# Run the embedding process with the following improvements:
# 1. Resume from batch 31
# 2. Use a smaller batch size of 10 instead of 100
# 3. Process a limited number of proteins per run to avoid long-running jobs
embed_proteins_low_memory(
    unique_proteins, 
    output_dir, 
    batch_size=100,  # Reduced from 100
    start_batch=0,  # Resume from batch 31
    max_proteins_per_run=10000  # Process a limited number before exiting
)

Will process from batch 1 to 97 (of 97 total)
Batch 1 already exists, skipping...
Batch 2 already exists, skipping...
Batch 3 already exists, skipping...
Batch 4 already exists, skipping...
Batch 5 already exists, skipping...
Batch 6 already exists, skipping...
Batch 7 already exists, skipping...
Batch 8 already exists, skipping...
Batch 9 already exists, skipping...
Batch 10 already exists, skipping...
Batch 11 already exists, skipping...
Batch 12 already exists, skipping...
Batch 13 already exists, skipping...
Batch 14 already exists, skipping...
Batch 15 already exists, skipping...
Batch 16 already exists, skipping...
Batch 17 already exists, skipping...
Batch 18 already exists, skipping...
Batch 19 already exists, skipping...
Batch 20 already exists, skipping...
Batch 21 already exists, skipping...
Batch 22 already exists, skipping...
Batch 23 already exists, skipping...
Batch 24 already exists, skipping...
Batch 25 already exists, skipping...
Batch 26 already exists, skipping...
B

Processing batch 31/97:   3%|▎         | 3/100 [00:00<00:24,  4.00it/s]

Truncating sequence for Q8WZ42 from 34350 to 1500
Truncating sequence for Q6YHU6 from 1953 to 1500


Processing batch 31/97:  43%|████▎     | 43/100 [00:09<00:12,  4.57it/s]

Truncating sequence for Q92508 from 2521 to 1500
Truncating sequence for P20929 from 6669 to 1500


...

Truncating sequence for Q15648 from 1581 to 1500


Processing batch 34/97:  83%|████████▎ | 83/100 [00:18<00:03,  4.50it/s]

Truncating sequence for P50748 from 2209 to 1500


Processing batch 34/97: 100%|██████████| 100/100 [00:22<00:00,  4.54it/s]

Saved batch 34 (100 proteins): 90.41 MB
Batch 35 already exists, skipping...
Batch 36 already exists, skipping...
Batch 37 already exists, skipping...
Batch 38 already exists, skipping...
Batch 39 already exists, skipping...
Batch 40 already exists, skipping...
Batch 41 already exists, skipping...
Batch 42 already exists, skipping...
Batch 43 already exists, skipping...
Batch 44 already exists, skipping...
Batch 45 already exists, skipping...
Batch 46 already exists, skipping...
Batch 47 already exists, skipping...
Batch 48 already exists, skipping...
Batch 49 already exists, skipping...
Batch 50 already exists, skipping...
Batch 51 already exists, skipping...
Batch 52 already exists, skipping...
Batch 53 already exists, skipping...
Batch 54 already exists, skipping...
Batch 55 already exists, skipping...
Batch 56 already exists, skipping...
Batch 57 already exists, skipping...
Batch 58 already exists, skipping...
Batch 59 already exists, skipping...
Batch 60 already exists, skipping..

In [20]:
import pickle
import torch
import os
from tqdm.auto import tqdm
import gc

def merge_embeddings(embeddings_dir, output_file=None):
    """
    Merge all protein embeddings from batch files into a single pickle file.
    
    Args:
        embeddings_dir: Directory containing the embedding batch files and manifest
        output_file: Path to save the merged embeddings (default: embeddings_merged.pkl in embeddings_dir)
    
    Returns:
        Path to the merged embeddings file
    """
    # Set default output file path if not provided
    if output_file is None:
        output_file = os.path.join(embeddings_dir, "embeddings_merged.pkl")
    
    # Load the manifest file
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    print(f"Found {len(manifest)} proteins in manifest")
    
    # Get unique batch files
    batch_files = set(manifest.values())
    print(f"Found {len(batch_files)} batch files")
    
    # Create a dictionary to hold all embeddings
    all_embeddings = {}
    
    # Track file size for reporting
    total_loaded_bytes = 0
    
    # Process each batch file
    for batch_file in tqdm(batch_files, desc="Loading batch files"):
        batch_path = os.path.join(embeddings_dir, batch_file)
        
        # Get file size
        file_size = os.path.getsize(batch_path)
        total_loaded_bytes += file_size
        
        # Load batch embeddings
        batch_embeddings = torch.load(batch_path)
        
        # Add embeddings to master dictionary
        all_embeddings.update(batch_embeddings)
        
        # Clear memory
        del batch_embeddings
        gc.collect()
    
    print(f"Loaded {len(all_embeddings)} protein embeddings")
    print(f"Total loaded: {total_loaded_bytes/1024/1024/1024:.2f} GB")
    
    # Save the merged embeddings
    print(f"Saving merged embeddings to {output_file}...")
    with open(output_file, "wb") as f:
        pickle.dump(all_embeddings, f)
    
    # Get size of the merged file
    merged_size = os.path.getsize(output_file)
    print(f"Merged file size: {merged_size/1024/1024/1024:.2f} GB")
    
    return output_file

def merge_embeddings_compressed(embeddings_dir, output_file=None):
    """
    Merge all protein embeddings from batch files into a single compressed pickle file.
    Uses highest compression level to minimize size.
    
    Args:
        embeddings_dir: Directory containing the embedding batch files and manifest
        output_file: Path to save the merged embeddings (default: embeddings_merged_compressed.pkl in embeddings_dir)
    
    Returns:
        Path to the merged embeddings file
    """
    import gzip
    
    # Set default output file path if not provided
    if output_file is None:
        output_file = os.path.join(embeddings_dir, "embeddings_merged_compressed.pkl")
    
    # Load the manifest file
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    print(f"Found {len(manifest)} proteins in manifest")
    
    # Get unique batch files
    batch_files = set(manifest.values())
    print(f"Found {len(batch_files)} batch files")
    
    # Create a dictionary to hold all embeddings
    all_embeddings = {}
    
    # Track file size for reporting
    total_loaded_bytes = 0
    
    # Process each batch file
    for batch_file in tqdm(batch_files, desc="Loading batch files"):
        batch_path = os.path.join(embeddings_dir, batch_file)
        
        # Get file size
        file_size = os.path.getsize(batch_path)
        total_loaded_bytes += file_size
        
        # Load batch embeddings
        batch_embeddings = torch.load(batch_path)
        
        # Add embeddings to master dictionary
        all_embeddings.update(batch_embeddings)
        
        # Clear memory
        del batch_embeddings
        gc.collect()
    
    print(f"Loaded {len(all_embeddings)} protein embeddings")
    print(f"Total loaded: {total_loaded_bytes/1024/1024/1024:.2f} GB")
    
    # Save the merged embeddings with compression
    print(f"Saving compressed merged embeddings to {output_file}...")
    with gzip.open(output_file, "wb", compresslevel=9) as f:
        pickle.dump(all_embeddings, f)
    
    # Get size of the merged file
    merged_size = os.path.getsize(output_file)
    print(f"Merged compressed file size: {merged_size/1024/1024/1024:.2f} GB")
    print(f"Compression ratio: {merged_size/total_loaded_bytes:.2f}")
    
    return output_file

# Usage example:
embeddings_dir = 'data/medium_set/embeddings'

# Choose one of these options:

# Option 1: Basic merge - faster but larger file
merged_file = merge_embeddings(embeddings_dir)
print(f"Merged embeddings saved to: {merged_file}")

# Option 2: Merge with maximum compression - slower but smaller file
# merged_compressed_file = merge_embeddings_compressed(embeddings_dir)
# print(f"Compressed merged embeddings saved to: {merged_compressed_file}")

Found 9647 proteins in manifest
Found 97 batch files


Loading batch files: 100%|██████████| 97/97 [00:33<00:00,  2.86it/s]


Loaded 9647 protein embeddings
Total loaded: 9.23 GB
Saving merged embeddings to data/medium_set/embeddings/embeddings_merged.pkl...
Merged file size: 9.23 GB
Merged embeddings saved to: data/medium_set/embeddings/embeddings_merged.pkl


In [23]:
protein_embeddings = pickle.load(open('data/medium_set/embeddings/embeddings_merged.pkl', 'rb'))


dict_keys(['P20749', 'Q6UXS0', 'Q14241', 'Q96G28', 'P47736', 'O75943', 'Q7L3B6', 'Q8N7C3', 'Q8TDH9', 'Q9BQD3', 'Q13698', 'Q06136', 'Q96DM3', 'O14508', 'Q8IY26', 'P78527', 'Q93079', 'Q13564', 'P31153', 'Q9BQ50', 'Q86X59', 'Q99613', 'Q9BVW6', 'P81274', 'Q13113', 'P54277', 'Q13308', 'Q86Y46', 'P34897', 'O75146', 'O15155', 'Q05519', 'O95154', 'Q9BZI1', 'Q9H8L6', 'P04553', 'Q9BU64', 'Q14789', 'Q9BUX1', 'P49903', 'Q9Y365', 'P16422', 'Q6ZMI0', 'Q6Q4G3', 'O43704', 'Q7RTU0', 'Q9NZN5', 'P11532', 'Q53F39', 'Q14008', 'Q5SQN1', 'O14604', 'Q9UBV8', 'Q16558', 'P60900', 'O60678', 'Q8IWE4', 'Q6DD88', 'Q8NBB4', 'Q9BVM2', 'Q9NT62', 'Q6L8H4', 'P01011', 'Q6ZV65', 'Q5JRA6', 'Q96T75', 'Q4G1C9', 'Q9NVV4', 'P05204', 'Q6UY14', 'Q6V702', 'O60930', 'Q8IZQ1', 'P14618', 'Q8N488', 'P18847', 'Q9UHV8', 'P45877', 'Q9UBP6', 'Q9UIE0', 'Q9H1K1', 'P62269', 'Q96Q40', 'P0CG04', 'Q02094', 'O94889', 'Q2M3C7', 'P56945', 'P01019', 'Q8IV36', 'Q96CD0', 'O60830', 'O14931', 'Q5TYM5', 'P16104', 'Q8IUE6', 'P25789', 'P36954', 'O43918',

In [27]:
for key, value in protein_embeddings.items():
    print(f"Protein ID: {key}, Embedding shape: {value.shape}")

Protein ID: P20749, Embedding shape: torch.Size([1, 456, 960])
Protein ID: Q6UXS0, Embedding shape: torch.Size([1, 138, 960])
Protein ID: Q14241, Embedding shape: torch.Size([1, 800, 960])
Protein ID: Q96G28, Embedding shape: torch.Size([1, 344, 960])
Protein ID: P47736, Embedding shape: torch.Size([1, 665, 960])
Protein ID: O75943, Embedding shape: torch.Size([1, 683, 960])
Protein ID: Q7L3B6, Embedding shape: torch.Size([1, 339, 960])
Protein ID: Q8N7C3, Embedding shape: torch.Size([1, 389, 960])
Protein ID: Q8TDH9, Embedding shape: torch.Size([1, 189, 960])
Protein ID: Q9BQD3, Embedding shape: torch.Size([1, 178, 960])
Protein ID: Q13698, Embedding shape: torch.Size([1, 1502, 960])
Protein ID: Q06136, Embedding shape: torch.Size([1, 334, 960])
Protein ID: Q96DM3, Embedding shape: torch.Size([1, 659, 960])
Protein ID: O14508, Embedding shape: torch.Size([1, 200, 960])
Protein ID: Q8IY26, Embedding shape: torch.Size([1, 297, 960])
Protein ID: P78527, Embedding shape: torch.Size([1, 15